In [ ]:
# Updated by Shubham Kumar
# This is final Notebook for this Problem.
# Using VarianceThresold for feature Selection
# Using StratifiedKFold
# Using QuadraticDiscriminantAnalysis Model

In [36]:
import pandas as pd
import numpy as np

In [42]:
from tqdm import tqdm_notebook
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [66]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
def acc(a,b):
    print('Accuracy(test Data): ',accuracy_score(a,b))
    #print('Accuracy(train Data):',accuracy_score(a,b))
    print('Confusion Matrix: \n',pd.DataFrame(confusion_matrix(a,b)))
    print('Classification Repost:\n',classification_report(a,b))

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [96]:
cols = train.columns
cols = list(cols)[1:-1]
cols.remove('wheezy-copper-turtle-magic')
len(cols)

255

In [99]:
tr_target = np.zeros(len(train))
preds = np.zeros(len(test))

In [102]:
for i in tqdm_notebook(range(512)):

    train2 = train[train['wheezy-copper-turtle-magic']==i]
    test2 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train2.index; idx2 = test2.index
    train2.reset_index(drop=True,inplace=True)

    data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
    data2 = VarianceThreshold(threshold=2).fit_transform(data[cols])

    train3 = data2[:train2.shape[0]]; test3 = data2[train2.shape[0]:]

    skf = StratifiedKFold(n_splits=11, random_state=42)
    for train_index, test_index in skf.split(train2, train2['target']):

        clf = QuadraticDiscriminantAnalysis(0.1)
        clf.fit(train3[train_index,:],train2.loc[train_index]['target'])
        tr_target[idx1[test_index]] = clf.predict(train3[test_index,:])
        preds[idx2] += clf.predict_proba(test3)[:,1] / skf.n_splits

In [101]:
auc = roc_auc_score(train['target'], tr_target)
print(f'AUC: {auc:.5}')

AUC: 0.9649


In [103]:
acc(train['target'],tr_target)

Accuracy(test Data):  0.9262199401855469
Confusion Matrix: 
         0       1
0  121302    9711
1    9630  121501
Classification Repost:
               precision    recall  f1-score   support

           0       0.93      0.93      0.93    131013
           1       0.93      0.93      0.93    131131

    accuracy                           0.93    262144
   macro avg       0.93      0.93      0.93    262144
weighted avg       0.93      0.93      0.93    262144



In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub['target'] = preds
#sub.to_csv('submission.csv', index=False)